In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
from skrl.envs.wrappers.torch import wrap_env
from skrl.memories.torch import RandomMemory
from skrl.models.torch import DeterministicMixin, Model, MultiCategoricalMixin
from skrl.trainers.torch import ParallelTrainer
from skrl.utils import set_seed
from tsl.TimeXer import Model as TimeXer

from preprocess import preprocess_timexer
from skrl.utils.spaces.torch import unflatten_tensorized_space

set_seed(42)

gym.register(
    id="MultiDatasetDiscretedTradingEnv",
    entry_point="timexer_env:MultiDatasetDiscretedTradingEnv",
    disable_env_checker=True,
)

In [2]:
env_cfg = dict(
    id="MultiDatasetDiscretedTradingEnv",
    dataset_dir="./data/futures/5m/2024/12/*.pkl",
    preprocess=preprocess_timexer,
    positions=[-1, 0, 1],
    multiplier=range(1, 51),
    trading_fees=0.001,
    borrow_interest_rate=0.003,
    portfolio_initial_value=1e3,
    max_episode_duration="max",
    verbose=0,
    window_size=96,
    btc_index=True,
)

In [ ]:
env = gym.make(**env_cfg)
obs = env.observation_space
env = wrap_env(env, wrapper="gymnasium")

In [4]:
device = env.device
replay_buffer_size = 1024 * 1 * env.num_envs
memory_size = int(replay_buffer_size / env.num_envs)

In [5]:
class SharedTSM(MultiCategoricalMixin, DeterministicMixin, Model):
    def __init__(
        self,
        observation_space,
        action_space,
        device,
        clip_actions=False,
        unnormalized_log_prob=True,
        reduction="sum",
        tsm_config=None,
    ):
        Model.__init__(self, observation_space, action_space, device)
        MultiCategoricalMixin.__init__(self, unnormalized_log_prob, reduction)
        DeterministicMixin.__init__(self, clip_actions)

        self._shared_features = None
        self._fusion = None

        # TimeXer 구성
        self.tsm = TimeXer(tsm_config)

        self.num_infos = 5

        # Shared tail
        self.shared_tail = nn.Sequential(
            nn.Linear(24 * 5 + self.num_infos, 32),
            nn.LayerNorm(32),
            nn.GELU(),
            nn.Dropout(0.2),
        )

        self.policy_head = nn.Sequential(nn.Linear(32, self.num_actions))
        self.value_head = nn.Sequential(nn.Linear(32, 1))

    def act(self, inputs, role):
        if role == "policy":
            return MultiCategoricalMixin.act(self, inputs, role)
        elif role == "value":
            return DeterministicMixin.act(self, inputs, role)

    def compute(self, inputs, role):
        states = unflatten_tensorized_space(obs, inputs["states"])
        features = states["features"]  # (batch_size, seq_length, num_features)
        infos = states["infos"]

        if role == "policy":
            ohlcv = features[:, :, 2:7]
            btc_btcdom = features[:, :, 0:2]
            tsm_output = self.tsm.forward(ohlcv, btc_btcdom, None, None)

            f = torch.cat([torch.flatten(tsm_output, 1), infos], dim=-1)
            self._fusion = self.shared_tail(f)

            actions = self.policy_head(self._fusion)

            return actions, {}

        elif role == "value":
            if self._fusion is None:
                ohlcv = features[:, :, 2:7]
                btc_btcdom = features[:, :, 0:2]
                tsm_output = self.tsm.forward(ohlcv, btc_btcdom, None, None)

                f = torch.cat([torch.flatten(tsm_output, 1), infos], dim=-1)
                fusion = self.shared_tail(f)
            else:
                fusion = self._fusion

            self._fusion = None

            value = self.value_head(fusion)
            return value, {}


In [6]:
class Config:
    def __init__(self, **entries):
        self.__dict__.update(entries)

tsm_config = {
    "task_name": "long_term_forecast",  # 수행 작업 이름 (논문 추천값: 'long_term_forecast')
    "seq_len": 96,  # 입력 시퀀스 길이 (논문 추천값: 96)
    "pred_len": 24,  # 예측할 출력 시퀀스 길이 (논문 추천값: 24)
    "d_model": 128,  # 모델 임베딩 차원 (논문 추천값: 512)
    "enc_in": 5,  # 입력 피처 수 (데이터에 따라 다름, 예: 7)
    "patch_len": 16,  # 패치 길이 (논문 추천값: 16)
    "dropout": 0.1,  # 드롭아웃 비율 (논문 추천값: 0.1)
    "n_heads": 8,  # 멀티헤드 어텐션의 헤드 수 (논문 추천값: 8)
    "e_layers": 3,  # Encoder 레이어 수 (논문 추천값: 3)
    "factor": 5,  # 어텐션 스파스니스 제어 (논문 추천값: 5)
    "d_ff": 512,  # 피드포워드 레이어 차원 (d_model의 4배, 논문 추천값: 2048)
    "embed": "timeF",
    "freq": "t",  # 시간 정보의 주파수 (논문 추천값: 'h'는 시간 단위)
    "use_norm": False,  # 정규화 사용 여부 (논문 추천값: True)
    "activation": "gelu",  # 활성화 함수 (논문 추천값: 'gelu')
    "features": "M",  # 입력 데이터 타입 (단일 변수: 'S', 다중 변수: 'M', 논문 추천: 'M')
}

tsm_config = Config(**tsm_config)

In [7]:
models = {}
models["policy"] = SharedTSM(
    env.observation_space,
    env.action_space,
    device,
    tsm_config=tsm_config,
)
models["value"] = models["policy"]

for model in models.values():
    model.init_parameters(method_name="normal_", mean=0.0, std=0.1)


In [8]:
from skrl.agents.torch.ppo import PPO_DEFAULT_CONFIG
from skrl.resources.schedulers.torch import KLAdaptiveLR
import warnings

from skrl.agents.torch.ppo import PPO

cfg = PPO_DEFAULT_CONFIG.copy()
cfg["rollouts"] = memory_size
cfg["learning_epochs"] = 16
cfg["mini_batches"] = 8
cfg["discount_factor"] = 0.99
cfg["learning_rate"] = 5e-4
cfg["learning_rate_scheduler"] = KLAdaptiveLR
cfg["learning_rate_scheduler_kwargs"] = {
    "kl_threshold": 0.01,
    "min_lr": 1e-7,
    "max_lr": 1e-3,
}
cfg["mixed_precision"] = True

cfg["experiment"]["write_interval"] = 20000
cfg["experiment"]["checkpoint_interval"] = 100000
cfg["experiment"]["directory"] = "runs/torch/tsm_run"

In [9]:
class PPO_EVAL(PPO):
    def act(self, states: torch.Tensor, timestep: int, timesteps: int) -> torch.Tensor:
        if timestep < self._random_timesteps:
            return self.policy.random_act({"states": self._state_preprocessor(states)}, role="policy")

        with torch.autocast(device_type=self._device_type, enabled=self._mixed_precision):
            actions, log_prob, outputs = self.policy.act({"states": self._state_preprocessor(states)}, role="policy")
            self._current_log_prob = log_prob

        dist = self.policy.distribution(role="policy")
        actions = dist.mean

        log_prob = dist.log_prob(actions)
        return actions, log_prob, outputs

In [10]:
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)

agent = PPO(
    models=models,
    cfg=cfg,
    observation_space=env.observation_space,
    action_space=env.action_space,
    device=device,
)
path = "25-01-02_11-26-13-255664_PPO"
path = "25-01-03_16-16-18-577452_PPO"
agent.load(f"/home/pitin/Desktop/hp/runs/torch/tsm/{path}/checkpoints/agent_30000.pt")
cfg_trainer = {"timesteps": 10000, "headless": True}
trainer = ParallelTrainer(cfg=cfg_trainer, env=env, agents=[agent])

In [ ]:
trainer.eval()

In [ ]:
states, infos = env.reset()
timestep = 0
timesteps = 3000
terminated = torch.tensor([[False] * env.num_envs])
print(infos["portfolio_valuation"])
# while (not terminated.any()):
for t in range(timesteps):
    agent.pre_interaction(timestep=t, timesteps=timesteps)

    with torch.no_grad():
        actions = agent.act(states, timestep=t, timesteps=timesteps)[0]
        next_states, rewards, terminated, truncated, infos = env.step(actions)
        env.render()

    super(type(agent), agent).post_interaction(timestep=t, timesteps=timesteps)

    if env.num_envs > 1:
        states = next_states
    else:
        if terminated.any() or truncated.any():
            with torch.no_grad():
                states, infos = env.reset()
        else:
            states = next_states
print(infos["portfolio_valuation"])
# env.save_for_render()

In [ ]:
from gym_trading_env.renderer import Renderer
renderer = Renderer(render_logs_dir="render_logs")
renderer.run()